**1. Quantum Simulator (Statevector + Gates)**

In [1]:
import numpy as np

# ===========================
# Quantum Simulator Utilities
# ===========================

def ket0():
    return np.array([1, 0], dtype=complex)

def ket1():
    return np.array([0, 1], dtype=complex)

def tensor(*states):
    out = states[0]
    for s in states[1:]:
        out = np.kron(out, s)
    return out

# --- Basic Gates ---
I = np.eye(2, dtype=complex)

X = np.array([[0, 1],
              [1, 0]], dtype=complex)

Z = np.array([[1, 0],
              [0, -1]], dtype=complex)

H = (1/np.sqrt(2)) * np.array([[1, 1],
                               [1, -1]], dtype=complex)

# -----------------------------
# Apply 1-qubit gate to n-qubit state
# -----------------------------
def apply_single_qubit_gate(state, gate, target, n_qubits):
    op = 1
    for i in range(n_qubits):
        op = np.kron(op, gate if i == target else I)
    return op @ state

# -----------------------------
# Apply CNOT to n-qubit state
# -----------------------------
def apply_cnot(state, control, target, n_qubits):
    dim = 2 ** n_qubits
    new_state = np.zeros(dim, dtype=complex)

    for i in range(dim):
        bits = [(i >> k) & 1 for k in range(n_qubits)]
        j = i

        if bits[control] == 1:
            bits[target] ^= 1
            j = 0
            for k in range(n_qubits):
                j |= (bits[k] << k)

        new_state[j] += state[i]

    return new_state

# -----------------------------
# Measure a single qubit
# -----------------------------
def measure(state, n_qubits, target):
    probabilities = np.abs(state)**2

    # probability of measuring |0> on target
    prob0 = 0.0
    for idx in range(len(state)):
        bit = (idx >> target) & 1
        if bit == 0:
            prob0 += probabilities[idx]

    # actual outcome
    result = 0 if np.random.rand() < prob0 else 1

    # collapse state
    post = np.zeros_like(state)
    for idx in range(len(state)):
        bit = (idx >> target) & 1
        if bit == result:
            post[idx] = state[idx]

    return result, post / np.linalg.norm(post)


**2. Random-Bit Generator + Superdense Coding Protocol**

In [2]:
# ====================================================
# Superdense Coding Protocol + Random Bit Generator
# ====================================================

# We will use 3 qubits:
# q0 = random bit generator (will produce c and d)
# q1 = Alice's qubit (entangled)
# q2 = Bob's qubit (entangled)

def generate_random_bit():
    """
    Uses q0 to produce one random bit by:
    1. Putting q0 in |0>
    2. Applying H
    3. Measuring
    """
    state = tensor(ket0(), ket0(), ket0())  # full 3-qubit system

    # Apply H to q0
    state = apply_single_qubit_gate(state, H, target=0, n_qubits=3)

    # Measure q0 => random bit
    bit, state = measure(state, n_qubits=3, target=0)
    return bit


def superdense_single_shot():
    """
    Runs ONE full iteration of the protocol:
    1. Randomly generate classical bits c and d using q0.
    2. Prepare Bell pair between Alice (q1) and Bob (q2).
    3. Alice encodes X^c Z^d on q1.
    4. Bob decodes using CNOT + H.
    5. Both measure to get (decoded_c, decoded_d).
    """

    # ---- STEP 1: Random bits c and d ----
    c = generate_random_bit()
    d = generate_random_bit()

    # Reset full state to |000>
    state = tensor(ket0(), ket0(), ket0())

    # ---- STEP 2: Prepare Bell pair on q1 & q2 ----
    # Apply H to Alice's qubit q1
    state = apply_single_qubit_gate(state, H, target=1, n_qubits=3)

    # CNOT: q1 -> q2
    state = apply_cnot(state, control=1, target=2, n_qubits=3)

    # ---- STEP 3: Alice encodes X^c Z^d on qubit q1 ----
    if d == 1:  # Z first (commute but standard order)
        state = apply_single_qubit_gate(state, Z, target=1, n_qubits=3)
    if c == 1:  # X
        state = apply_single_qubit_gate(state, X, target=1, n_qubits=3)

    # ---- STEP 4: Bob performs decoding ----
    # CNOT: q1 -> q2
    state = apply_cnot(state, control=1, target=2, n_qubits=3)

    # Apply H on q1
    state = apply_single_qubit_gate(state, H, target=1, n_qubits=3)

    # ---- STEP 5: Measure decoded bits ----
    # IMPORTANT:
    # - Alice's measurement (q1) gives decoded Z-bit = d
    # - Bob's measurement (q2) gives decoded X-bit = c

    decoded_d, state = measure(state, n_qubits=3, target=1)  # Alice
    decoded_c, state = measure(state, n_qubits=3, target=2)  # Bob

    return {
        "c_generated": c,
        "d_generated": d,
        "decoded_c": decoded_c,
        "decoded_d": decoded_d
    }


**3. Run Many Shots + Verify Correctness**

In [3]:
import pandas as pd

# ==========================================
# Run many shots of superdense coding
# ==========================================

def run_experiment(shots=50):
    results = []
    
    for _ in range(shots):
        res = superdense_single_shot()
        results.append(res)
    
    df = pd.DataFrame(results)

    # Correctness check:
    # decoded_d == d_generated  (Alice)
    # decoded_c == c_generated  (Bob)
    df["match"] = (
        (df["decoded_c"] == df["c_generated"]) &
        (df["decoded_d"] == df["d_generated"])
    )

    return df


# Run experiment
df = run_experiment(shots=100)   # You can increase to 1000+ if you want

# Show results
print(df)

# Summary
print("\nCorrect transmissions:", df['match'].sum(), "/", len(df))
if df['match'].all():
    print("✓ All bits successfully transmitted! Superdense coding works.")
else:
    print("✗ Some mismatches occurred.")


    c_generated  d_generated  decoded_c  decoded_d  match
0             0            0          0          0   True
1             0            0          0          0   True
2             0            0          0          0   True
3             0            0          0          0   True
4             0            0          0          0   True
..          ...          ...        ...        ...    ...
95            0            0          0          0   True
96            0            0          0          0   True
97            0            0          0          0   True
98            0            0          0          0   True
99            0            0          0          0   True

[100 rows x 5 columns]

Correct transmissions: 100 / 100
✓ All bits successfully transmitted! Superdense coding works.


**Inference**

- Superdense coding transmitted 2 classical bits using 1 qubit with pre-shared entanglement.

- Random bits c and d were generated using an extra qubit.

- Alice encoded via X^c Z^d, Bob decoded with CNOT + H; all bits matched perfectly.

- Confirms entanglement increases classical channel capacity in a noise-free simulation.